# CVRP-ACO Demo: Solving Constrained Vehicle Routing Problems

This notebook demonstrates how to use Ant Colony Optimization (ACO) to solve Constrained Vehicle Routing Problems (CVRP).

**🎓 This notebook serves as a front page for the Gumroad CVRP-ACO course**

## What is CVRP?

CVRP involves delivering goods from a central depot to multiple customers using vehicles with capacity constraints (pallets and weight).

## What is ACO?

ACO is a metaheuristic inspired by ant foraging behavior. Ants deposit pheromones on paths, influencing future ants to follow shorter routes.

## Steps in this Demo:

1. Set up data and API keys
2. Understand the pre-built data files
3. Run ACO solver with your chosen parameters
4. Visualize results


# Clone repository and setup (run this first!)

In [ ]:
# Clean setup - remove any existing directory and clone fresh
!rm -rf /content/cvrp-routing-solver 2>/dev/null || true
!git clone https://github.com/onRubik/cvrp-routing-solver.git
%cd /content/cvrp-routing-solver
!pip install -r requirements.txt

In [ ]:
# Import modules
import sys
import os
import pandas as pd
import sqlite3
from pathlib import Path

# Setup paths
sys.path.append('src')

# Check if we're in the right directory
if not Path('data/input_small.csv').exists():
    print("❌ Error: data/input_small.csv not found. Please ensure you're in the cvrp-routing-solver directory.")
    sys.exit(1)

print("✅ Environment set up successfully!")
print(f"📁 Current working directory: {os.getcwd()}")

# Step 1: Understand the Pre-built Data Files

This system comes with pre-computed data to avoid API calls and focus on the CVRP solving algorithm.

## 📄 input_small.csv
- Contains 20 location IDs (ways, nodes, and relations from OpenStreetMap)
- These represent store locations in Manhattan, NYC
- One location (way/1180028681) is designated as the Distribution Center

## 📄 geo_points.csv
- Geographic coordinates for all locations
- Realistic delivery attributes assigned to each store:
  - **delivery_freq_per_week**: How often this store needs deliveries (1-7 times/week)
  - **pall_avg**: Average number of pallets required per delivery
  - **lbs_avg**: Average weight in pounds per delivery

## 📄 geo_permutations.csv
- Pre-calculated distances between ALL pairs of locations
- Distances are in meters (real driving distances from Open Route Service)
- This avoids the need for API calls and rate limits

In [ ]:
# Load and examine the data files

print("📄 INPUT LOCATIONS (input_small.csv):")
input_points = pd.read_csv('data/input_small.csv', header=None, names=['location_id'])
print(f"Total locations: {len(input_points)}")
print(input_points.head(10))
print("\n")

print("📍 GEOGRAPHIC POINTS (geo_points.csv):")
geo_points = pd.read_csv('data/geo_points.csv')
print(f"Total points with attributes: {len(geo_points)}")
print("Columns:", list(geo_points.columns))
print(geo_points.head())
print("\n")

print("🗺️ DISTANCE MATRIX (geo_permutations.csv):")
geo_perms = pd.read_csv('data/geo_permutations.csv')
print(f"Total distance pairs: {len(geo_perms):,}")
print(f"Expected pairs: {len(geo_points) * (len(geo_points) - 1):,}")
print(geo_perms.head())
print("\n")

# Show distribution center info
dc_info = geo_points[geo_points['id_p'] == 'way/1180028681']
print("🏭 DISTRIBUTION CENTER:")
print(dc_info)

# Step 2: Choose Origin Point (Distribution Center)

- **Default**: way/1180028681 (Distribution Center)
- **Options**: Any of the 20 locations in input_small.csv
- The origin is where tractors start and return to after deliveries
- Enter your choice below:

In [ ]:
# Choose origin point
print("Available origin points:")
for i, row in geo_points.iterrows():
    marker = "🏭" if row['id_p'] == 'way/1180028681' else "🏪"
    print(f"{marker} {row['id_p']}: {row['name']}")

# Default origin
origin_point = 'way/1180028681'  # Distribution Center

print(f"\n🎯 Using default origin: {origin_point} (Distribution Center)")
print("💡 To use a different origin, modify the origin_point variable above")

# Validate origin exists
if origin_point not in geo_points['id_p'].values:
    print(f"❌ Error: Origin point {origin_point} not found in geo_points.csv")
    sys.exit(1)

print(f"✅ Origin point validated: {origin_point}")

# Step 3: Set Vehicle Constraints

Configure the tractor capacity constraints:
- **max_pall**: Maximum pallets per tractor
- **max_lbs**: Maximum weight in pounds per tractor

These constraints determine how many stores each tractor can visit before returning to the distribution center.

In [ ]:
# Vehicle constraints
max_pall = 22  # Maximum pallets per tractor
max_lbs = 38000  # Maximum weight per tractor (38,000 lbs = ~17 tons)

print(f"🚛 Tractor constraints:")
print(f"   Max pallets: {max_pall}")
print(f"   Max weight: {max_lbs:,} lbs")

# Show total demand
stores = geo_points[geo_points['id_p'] != origin_point]
total_pall = stores['pall_avg'].sum()
total_lbs = stores['lbs_avg'].sum()

print(f"\n📦 Total delivery requirements:")
print(f"   Total pallets needed: {total_pall}")
print(f"   Total weight needed: {total_lbs:,.0f} lbs")

min_tractors_pall = int(total_pall // max_pall) + 1
min_tractors_lbs = int(total_lbs // max_lbs) + 1
min_tractors = max(min_tractors_pall, min_tractors_lbs)

print(f"\n🔢 Minimum tractors needed: {min_tractors}")
print(f"   (Based on pallet capacity: {min_tractors_pall}, weight capacity: {min_tractors_lbs})")

# Step 4: Set Up Database and Load Data

Create SQLite database and load the CSV data.

In [ ]:
# Set up database
db_path = 'data/cvrp_demo.db'

# Remove existing database if it exists
if Path(db_path).exists():
    Path(db_path).unlink()
    print("🗑️ Removed existing database")

# Create database connection
con = sqlite3.connect(db_path)
cur = con.cursor()

# Create tables
cur.execute('''
CREATE TABLE geo_points (
    id_p TEXT PRIMARY KEY,
    name TEXT,
    lat NUMERIC,
    lon NUMERIC,
    delivery_freq_per_week NUMERIC,
    pall_avg INTEGER,
    lbs_avg NUMERIC
)
''')

cur.execute('''
CREATE TABLE geo_permutations (
    perm TEXT PRIMARY KEY,
    id_1 TEXT,
    id_2 TEXT,
    name_1 TEXT,
    name_2 TEXT,
    lat_id_1 NUMERIC,
    lon_id_1 NUMERIC,
    lat_id_2 NUMERIC,
    lon_id_2 NUMERIC,
    distance NUMERIC
)
''')

cur.execute('''
CREATE TABLE dvrp_set (
    id_set INTEGER PRIMARY KEY AUTOINCREMENT,
    dvrp_id TEXT,
    cluster_id INTEGER,
    cluster_name TEXT,
    point TEXT,
    sequence INTEGER
)
''')

cur.execute('''
CREATE TABLE dvrp_origin (
    dvrp_id TEXT PRIMARY KEY,
    dvrp_origin TEXT
)
''')

# Load data
geo_points.to_sql('geo_points', con, if_exists='append', index=False)
geo_perms.to_sql('geo_permutations', con, if_exists='append', index=False)

con.commit()
print(f"✅ Database created and populated with {len(geo_points)} points and {len(geo_perms)} distance pairs")
print(f"📁 Database saved to: {db_path}")

# Verify data
print("\n🔍 Verification:")
cursor = con.execute("SELECT COUNT(*) FROM geo_points")
print(f"   Points in database: {cursor.fetchone()[0]}")

cursor = con.execute("SELECT COUNT(*) FROM geo_permutations")
print(f"   Distance pairs in database: {cursor.fetchone()[0]}")

con.close()

# Step 5: Run ACO Solver

Now we'll solve the CVRP using Ant Colony Optimization.

**ACO Parameters:**
- **n_ants**: Number of ants per iteration (more = better exploration)
- **n_iterations**: Number of algorithm iterations (more = better solutions)
- **alpha**: Pheromone importance (higher = follow pheromone trails more)
- **beta**: Distance importance (higher = prefer shorter distances)
- **evaporation_rate**: How fast pheromones evaporate (0.1-0.9)
- **Q**: Pheromone deposit factor

In [ ]:
# Import and configure ACO solver
from src.solver.aco import ACOSolver

# ACO parameters
aco_params = {
    'n_ants': 30,
    'n_iterations': 50,
    'alpha': 1.0,
    'beta': 2.0,
    'evaporation_rate': 0.5,
    'Q': 1.0
}

print("🐜 ACO Parameters:")
for param, value in aco_params.items():
    print(f"   {param}: {value}")

# Create solver instance
solver = ACOSolver(
    dvrp_id='gumroad_demo',
    points_file='data/input_small.csv',
    db_path=db_path,
    origin=origin_point,
    max_pall=max_pall,
    max_lbs=max_lbs,
    **aco_params
)

print(f"\n🚀 Starting ACO solver with {len(stores)} stores...")
print("This may take 1-2 minutes depending on parameters...")

# Run solver
result = solver.solve()

print(f"\n✅ ACO completed: {result}")
print("\n📊 Results saved to database!")

# Step 6: View Solution Results

Let's examine the solution that was found.

In [ ]:
# Load and display results
con = sqlite3.connect(db_path)

# Get solution routes
routes_query = """
SELECT 
    cluster_id,
    cluster_name,
    point,
    sequence,
    gp.name as store_name,
    gp.pall_avg as pallets,
    gp.lbs_avg as weight_lbs
FROM dvrp_set ds
JOIN geo_points gp ON ds.point = gp.id_p
WHERE dvrp_id = 'gumroad_demo'
ORDER BY cluster_id, sequence
"""

routes_df = pd.read_sql_query(routes_query, con)

if routes_df.empty:
    print("❌ No solution found. Check solver output above.")
else:
    print("🚛 SOLUTION ROUTES:")
    
    # Group by tractor
    tractors = routes_df.groupby('cluster_id')
    
    for tractor_id, tractor_routes in tractors:
        tractor_name = tractor_routes['cluster_name'].iloc[0]
        total_pall = tractor_routes['pallets'].sum()
        total_weight = tractor_routes['weight_lbs'].sum()
        
        print(f"\n{tractor_name} (Tractor {tractor_id + 1}):")
        print(f"   Total pallets: {total_pall}, Total weight: {total_weight:,.0f} lbs")
        print("   Route:")
        
        # Build route string
        route_points = ["Distribution Center"]
        for _, stop in tractor_routes.iterrows():
            route_points.append(f"{stop['store_name']} ({stop['pallets']} pall, {stop['weight_lbs']:,.0f} lbs)")
        route_points.append("Distribution Center")
        
        for i, point in enumerate(route_points):
            prefix = f"   {i+1}." if i > 0 else "   0."
            print(f"{prefix} {point}")
    
    print(f"\n📊 SUMMARY:")
    print(f"   Total tractors used: {len(tractors)}")
    print(f"   Total stores served: {len(routes_df)}")
    print(f"   Average stores per tractor: {len(routes_df)/len(tractors):.1f}")

con.close()

# Step 7: Visualize Routes (Map)

Create an interactive map showing the delivery routes.

In [ ]:
# Create route visualization using proper function
from src.viz.plots import plot_routes_map

# Display routes correctly following database sequence
routes_map = plot_routes_map('gumroad_demo', db_path=db_path)
routes_map.show()

# 🎓 Next Steps - Learn More!

This demo shows the basics of CVRP solving with ACO. For the complete course covering:

- **Advanced ACO algorithms** and parameter tuning
- **Real-world logistics optimization** techniques
- **Integration with actual routing APIs**
- **Performance optimization** for large datasets
- **Custom constraint modeling**

**👉 Check out our Gumroad courses:**
- [CVRP Fundamentals Course](https://gumroad.com/your-course-link)
- [ACO Algorithm Deep Dive](https://gumroad.com/your-course-link)
- [Real-World Logistics Optimization](https://gumroad.com/your-course-link)

# Experiment Further

- Try different origin points
- Adjust vehicle capacity constraints
- Modify ACO parameters
- Add time windows or other constraints

Happy optimizing! 🚛✨